In [1]:
%matplotlib widget

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

# Accumulated products

In [2]:
# Odd navigation with gaps between pixels
# ds=xr.open_dataset('./LI_L2_Format_Familiarisation_for_Users/ARC/LI-2-AFR/W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-AFR--FD--CHK-BODY--ARC-NC4E_C_EUMT_20170616080608_L2PF_DEV_20130620000000_20130620001000_N__T_0001_0000.nc')
# ds=xr.open_dataset('./LI_L2_Format_Familiarisation_for_Users/DIS/LI-2-AFR/W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-AFR--FD--CHK-BODY--DIS-NC4E_C_EUMT_20160323082235_L2PF_DEV_20130620010000_20130620010030_N__T_0007_0001.nc')

# Good navigation
ds=xr.open_dataset('./LI_L2_TestData_SAFs_V1.0/L2_run_flashes_201306200700_201306200701/W_XX-EUMETSAT-DARMSTADT,IMG+SAT,MTGI1+LI-2-AFR--FD--CHK-BODY---NC4E_C_EUMT_20190116151947_L2PF_DEV_20130620070000_20130620070030_N__T_0043_0001.nc')



In [3]:
print(len(ds.variables.keys()))
print(len(ds.attrs.keys()))

11
70


In [4]:
# ds.time_coverage_start, ds.time_coverage_end
# ds.start_time, ds.end_time

In [5]:
plt.close('all')
fig = plt.figure()
plt.scatter(ds.x, ds.y, c=np.log10(ds.flash_radiance), s=36, marker='s')
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
# def min_sep(x):
    
#     delta = np.sort(np.abs(np.diff(x)))
#     nonzero = ~np.isclose(delta,0.0)
#     return delta[nonzero]

# sep_x = min_sep(ds.x)
# sep_y = min_sep(ds.y)
# print('min dx')
# for v in min_sep(ds.x)[0:10]:
#     print(v)
# print('min dy') 
# for v in min_sep(ds.y)[0:10]:
#     print(v)

In [7]:
def to_fixed_grid(c,r, delta_ew=5.58871526031607e-05, delta_sn=5.58871526031607e-05, lam_0=1.555618892700898e-1, phi_0=-1.555618892700898e-1):
    """
    c, r: column (west-east), row (south-north) pixel id
    delta_ew, delta_sn: fixed grid angle between each pixel (constant)
    lam_0, phi_0: azimuth (ew), elevation (ns) angles from the centre of the projection to the centre of the pixel in
        the first row and first column of the reference grid, respectively .
    
    "Let (r,c) be the coordinates row and column. Row and columns are counted increasingly
    when going from bottom to up (south to north) and left to right (west to east) and beginning at 1.
    Therefore, the South-West corner of the grid has coordinates (1,1). 
    The correspondence between the row and column position (r, c) and the :"
    
    w-e is azimuth and s-n is elevation in docs
    "Note that the E-W viewing angle (lma_0) does not correspond to the standard definition of azimuth,
    for an observation from the instrument perspective, which runs from negative to positive from West to East. 
    Instead, it runs from negative to positive from East to West. 
    
    The N-S viewing angle corresponds to the standard definition of elevation, for an observation from the instrument perspective."
    
    """
    lam_s = lam_0 - (c-1)*delta_ew
    phi_s = phi_0 + (r-1)*delta_sn    
    return lam_s, phi_s

def to_pixel_id(lam, phi, delta_ew=5.58871526031607e-05, delta_sn=5.58871526031607e-05, lam_0=1.555618892700898e-1, phi_0=-1.555618892700898e-1):
    c = -(lam - lam_0)/delta_ew + 1
    r =  (phi - phi_0)/delta_sn + 1
    return np.round(c).astype('int32'), np.round(r).astype('int32')

In [8]:
ds.x.attrs['valid_range']

array([   1, 5568], dtype=int16)

In [9]:
def pixel_valid_range(ds):
    # 1-indexed, so max value is the length of the 2D image array along that dimension
    try:
        # attribute is comma separated string
        x_valid_range = np.fromiter(map(int, ds.x.attrs['valid_range'].split(',')), dtype=int)
        y_valid_range = np.fromiter(map(int, ds.y.attrs['valid_range'].split(',')), dtype=int)
    except AttributeError:
        # attribute is already a 2-element integer array
        x_valid_range = ds.x.attrs['valid_range']
        y_valid_range = ds.y.attrs['valid_range']
    return x_valid_range, y_valid_range

def image_size(ds):
    x_valid_range, y_valid_range = pixel_valid_range(ds)    
    nx = x_valid_range[1]-x_valid_range[0]+1
    ny = y_valid_range[1]-y_valid_range[0]+1
    return nx, ny

def create_image(ds, name, dims=('ypixels', 'xpixels')):
    """ dims must be the y (north south) coordinate name, followed by the x name"""
    dtype = ds[name].dtype
    nx, ny = image_size(ds)
    a = np.empty((ny, nx), dtype=dtype)*np.nan
    
    x_coord, y_coord = to_fixed_grid(np.arange(nx), np.arange(ny))
#     print(y_coord.min())
    
    da = xr.DataArray(a, dims=dims, coords={dims[0]:y_coord, dims[1]:x_coord})
    
    col, row = to_pixel_id(ds.x, ds.y)
    xid, yid = row-1, col-1
    print(xid.min(), yid.min(), xid.max(), yid.max())
    da.data[yid, xid] = np.log10(ds.flash_radiance)
    return da

da = create_image(ds, 'flash_radiance')
ds['total_optical_energy'] = da

<xarray.DataArray 'y' ()>
array(535, dtype=int32) <xarray.DataArray 'x' ()>
array(888, dtype=int32) <xarray.DataArray 'y' ()>
array(5206, dtype=int32) <xarray.DataArray 'x' ()>
array(4465, dtype=int32)


In [10]:
# import metpy
# print(metpy.__version__)
# # dsmap=ds.metpy.parse_cf()
# ds = ds.metpy.assign_crs(ds.mtg_geos_projection.attrs)

In [11]:
ds.total_optical_energy

<xarray.DataArray 'total_optical_energy' (ypixels: 5568, xpixels: 5568)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Coordinates:
  * ypixels  (ypixels) float64 -0.1556 -0.1556 -0.1555 ... 0.1554 0.1555 0.1555
  * xpixels  (xpixels) float64 0.1556 0.1556 0.1555 ... -0.1554 -0.1555 -0.1555

In [12]:
# geo_crs = ds.total_optical_energy.metpy.cartopy_crs
plt.figure()
da.plot.imshow(origin='lower')#, transform=geo_crs)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Flash products

In [24]:
dsfl=ds=xr.open_dataset('./LI_L2_Format_Familiarisation_for_Users/DIS/LI-2-LFL/W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--CHK-BODY--DIS-NC4E_C_EUMT_20160323082235_L2PF_DEV_20130620010000_20130620010010_N__T_0007_0001.nc')

In [25]:
dsfl.time_coverage_start, dsfl.time_coverage_start

('20130620010000', '20130620010000')

In [26]:
dsfl

<xarray.Dataset>
Dimensions:           (flashes: 57)
Dimensions without coordinates: flashes
Data variables:
    flash_id          (flashes) uint32 ...
    latitude          (flashes) float32 ...
    longitude         (flashes) float32 ...
    radiance          (flashes) float32 ...
    flash_time        (flashes) datetime64[ns] ...
    number_of_events  (flashes) uint16 ...
    number_of_groups  (flashes) uint16 ...
    flash_duration    (flashes) uint16 ...
    flash_footprint   (flashes) uint16 ...
    flags             (flashes) int32 ...
Attributes:
    Conventions:                          
    summary:                              
    keywords:                             
    history:                              original generated file
    institution:                          EUMETSAT
    product_id:                           LI-2-LFL
    processing_level:                     2
    baseline_version:                     
    processor_version:                    
    algorithm_version:                    
    format_version:                       4A
    special_compression:                  
    subsetting:                           
    facility_or_tool:                     L2PF
    references:                           
    comment:                              Test data generated for user famili...
    instrument_configuration_id:          0
    instrument_configuration_id_version:  0
    mtg_name:                             
    alternative_name:                     
    purpose:                              
    title:                                W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI...
    location_indicator:                   XX-EUMETSAT-Darmstadt
    data_designator:                      IMG+SAT
    release_version:                      original
    processing_mode:                      N
    disposition_mode:                     T
    source:                               LI
    environment:                          DEV
    group_tag:                            MTI1_LI_2_LFL__2013_171_0007_original
    subsettable_groups:                   
    subsettable_groups_present:           
    format:                               NC4E
    geospatial_lat_min:                   -90.0
    geospatial_lat_max:                   +90.0
    geospatial_lon_min:                   -90.0
    geospatial_lon_max:                   +90.0
    cdm_datatype:                         
    component1:                           CHK
    component2:                           BODY
    component3:                           
    count_in_repeat_cycle:                0001
    coverage:                             FD
    creator_email:                        
    creator_institution:                  
    creator_name:                         
    creator_type:                         
    creator_url:                          
    data_source:                          LI
    date_created:                         20160323082235
    date_time_position:                   
    id:                                   
    keywords_vocabulary:                  
    license:                              
    linked_data:                          
    naming_authority:                     
    parent_data:                          
    platform:                             MTI1
    processed_count_in_repeat_cycle:      0001
    project:                              
    repeat_cycle_in_day:                  0007
    runtime_data:                         
    standard_name_vocabulary:             
    subtype:                              
    time_coverage_duration:               
    time_coverage_resolution:             
    time_position:                        
    time_coverage_start:                  20130620010000
    time_coverage_end:                    20130620010010
    type:                                 LFL

In [27]:
import cartopy.crs as ccrs
plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
gl = ax.gridlines(linestyle=":", draw_labels=True)
ax.scatter(dsfl.longitude, dsfl.latitude, c=np.log10(ds.radiance), s=36, marker='s', transform=ccrs.PlateCarree())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …